# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [2]:
#Holidays?

In [3]:
df = pd.read_csv('df_flights_final.csv')

#drop flight date and first column
df = df.drop(columns='Unnamed: 0')
df.drop("fl_date",axis=1, inplace=True)

In [4]:
df.drop(['dest_airport_id', 'tail_num', 'state'], axis = 1, inplace=True)

In [5]:
# random sample of 10000 rows
df_small = df.sample(n = 10000)
df_small.shape

(10000, 35)

In [6]:
y = df_small.arr_delay
df_small.drop("arr_delay",axis=1, inplace=True)
X = df_small

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [9]:
# features with dtype is object
categorical_columns = X_train.columns[X_train.dtypes == "object"]
categorical_columns

Index(['mkt_unique_carrier', 'op_unique_carrier', 'origin', 'dest'], dtype='object')

In [10]:
# Encode categorical features as a one-hot numeric array
#enc = OneHotEncoder(handle_unknown='ignore')
#transformed_columns = enc.fit_transform(X_train[['mkt_unique_carrier', 'op_unique_carrier', 'origin', 'dest']])

In [11]:
# Transform non-numerical labels to numerical labels

In [12]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [13]:
X_train = MultiColumnLabelEncoder(columns = ['origin_airport_id', 'mkt_unique_carrier', 'op_unique_carrier', 'origin', 'dest']).fit_transform(X_train)

In [14]:
X_train

,mkt_unique_carrier,op_unique_carrier,origin_airport_id,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,fl_day,...,dep_mean_hourly_delay,arr_mean_hourly_delay,dep_std_hourly_delay,arr_std_hourly_delay,mean_mkt_carrier_delay,mean_op_carrier_delay,std_mkt_carrier_delay,std_op_carrier_delay,mean_tail_num_arr_delay,std_tail_num_arr_delay
60817,3,7,281,281,251,1412,1640,208.0,1216.0,3,...,12.756996,8.215507,34.773392,37.863256,3.100823,2.427403,16.979937,14.086325,1.157895,25.235036
54321,8,21,261,261,202,1620,2020,180.0,1250.0,3,...,14.393480,11.125661,37.446605,43.969117,3.414305,3.152323,17.667574,15.536797,-1.000000,21.329613
114410,9,22,151,151,234,1800,1920,80.0,345.0,6,...,16.602144,10.184557,39.663138,42.358665,5.081818,5.081818,17.168188,17.168188,1.318182,20.800014
122518,3,12,269,269,63,1220,1429,69.0,308.0,6,...,9.213188,3.873598,30.555721,34.177841,3.100823,4.139578,16.979937,18.343566,26.055556,63.062501
131316,0,15,289,289,202,1541,1734,113.0,522.0,7,...,13.025412,8.215507,34.956169,37.863256,4.098024,2.936698,18.659873,14.080183,9.571429,24.905536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131497,0,17,244,244,54,1952,2111,79.0,213.0,7,...,19.120787,12.272941,42.202577,44.169176,4.098024,3.554984,18.659873,18.181722,23.439024,43.721876
12076,0,19,236,236,54,1618,1734,76.0,155.0,1,...,14.393480,8.215507,37.446605,37.863256,4.098024,3.243643,18.659873,15.827478,20.275862,61.609635
104845,3,7,280,280,13,600,856,116.0,674.0,6,...,-2.813004,-6.334801,7.026809,19.292029,3.100823,2.427403,16.979937,14.086325,-4.894737,14.552185
101176,8,18,80,80,70,1639,1740,61.0,120.0,5,...,14.393480,8.215507,37.446605,37.863256,3.414305,4.461860,17.667574,22.749432,9.920000,29.698373


In [15]:
#df_transformed_columns = pd.DataFrame(transformed_columns.todense())

In [16]:
#df_transformed_columns

In [17]:
# concatinating X_train and encoded values

In [18]:
#X_train = pd.concat([X_train.drop(columns = categorical_columns).reset_index(), df_transformed_columns], axis=1)

In [19]:
#X_train.drop("index",axis=1, inplace=True)

In [20]:
#X_train

In [21]:
#from sklearn.preprocessing import StandardScaler

In [22]:
#data_for_scaling = X_train[['crs_dep_time', 'crs_arr_time', 'arr_delay', 'crs_elapsed_time',
       #'distance', 'hour_of_day_dep', 'hour_of_day_arr',
       #'daily_arr_delay_mean', 'daily_carrier_delay_mean',
       #'daily_weather_delay_mean', 'daily_nas_delay_mean',
       #'daily_security_delay_mean', 'daily_late_aircraft_delay_mean',
       #'daily_arr_delay_std', 'daily_carrier_delay_std',
       #'daily_weather_delay_std', 'daily_nas_delay_std',
       #'daily_security_delay_std', 'daily_late_aircraft_delay_std',
       #'dep_mean_hourly_delay', 'arr_mean_hourly_delay',
       #'dep_std_hourly_delay', 'arr_std_hourly_delay',
       #'mean_mkt_carrier_delay', 'mean_op_carrier_delay',
       #'std_mkt_carrier_delay', 'std_op_carrier_delay',
       #'mean_tail_num_arr_delay']]
#data_for_scaling.head()

In [23]:
#scaling data
#scaler = StandardScaler()
#scaler.fit(data_for_scaling)

In [24]:
#scaled_data

In [25]:
# concatinating X_train and scaled values

#X_train = pd.concat([X_train.reset_index(), scaled_data]

In [26]:
#from sklearn.feature_selection import f_regression, SelectKBest
#skb = SelectKBest(f_regression, k=20)
#X_train = skb.fit_transform(X_train, y_train)

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [34]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=10000)

In [35]:
logreg.fit(X_train,y_train)

/Users/katrindar/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000)

In [ ]:
y_pred=logreg.predict(X_test)

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.